In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os #tüm dosyaları okuyabilmek için


# ikili değerde matriks oluşturmak için
from tensorflow.keras.utils import to_categorical
# verileri temizleme ve organize etmek için
from tensorflow.keras.preprocessing import image
#model ve katman eklemek için
from tensorflow.keras import models, layers

# Library for Transfer Learning
from tensorflow.keras.applications import resnet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

In [ ]:
import zipfile

# ZIP dosyasının adını belirtin
zip_dosya_adi = '/content/drive/MyDrive/Dataset.zip'

# ZIP dosyasını çıkarın
with zipfile.ZipFile(zip_dosya_adi, 'r') as zip_arsiv:
    zip_arsiv.extractall('/content/drive/MyDrive')  # Çıkarılan dosyaları saklamak için bir dizin belirtin


In [ ]:
testFile = '/content/drive/MyDrive/Dataset/TestSet'
classNames = os.listdir(testFile)
#image array değerleri içim
testArray=[]
#image labelları için
testLabel=[]

#bu döngü sayesinde tüm verileri istediğimiz boyuta getirebiliyoruz
for folder in os.listdir(testFile): #klasör içindeki tüm klasörleri döner
    imageList=os.listdir(testFile+"/"+folder)
    for imgName in imageList: #içerideki klasörün içindeki verilerin hepsini döner

        img=image.load_img(testFile+"/"+folder+"/"+imgName,target_size=(224,224))

        #imageleri arraye çevirmek için
        img=image.img_to_array(img)

        # Transfer Learning:
        img=resnet50.preprocess_input(img)

        # Appending the arrarys
        testArray.append(img) # appending image array
        testLabel.append(classNames.index(folder)) # appending class index to the array


In [ ]:
valFile = '/content/drive/MyDrive/Dataset/ValSet'
valClassNames = os.listdir(valFile)
#image array değerleri içim
valArray=[]
#image labelları için
valLabel=[]

#bu döngü sayesinde tüm verileri istediğimiz boyuta getirebiliyoruz
for folder in os.listdir(valFile): #klasör içindeki tüm klasörleri döner
    imageList=os.listdir(valFile+"/"+folder)
    for imgName in imageList: #içerideki klasörün içindeki verilerin hepsini döner

        img=image.load_img(valFile+"/"+folder+"/"+imgName,target_size=(224,224))

        #imageleri arraye çevirmek için
        img=image.img_to_array(img)

        # Transfer Learning:
        img=resnet50.preprocess_input(img)

        # Appending the arrarys
        valArray.append(img) # appending image array
        valLabel.append(valClassNames.index(folder)) # appending class index to the array


In [ ]:
realresnetModel=resnet50.ResNet50(weights='imagenet',input_shape=(224,224,3),include_top=False,pooling='avg')


In [ ]:
realresnetModel.summary()


In [ ]:
for layer in realresnetModel.layers[:-1]:
  realresnetModel.trainable = False

In [ ]:
pip install tensorflow


In [ ]:
from tensorflow.keras.models import Sequential, Model
realresnetModel = Model(inputs=realresnetModel.input, outputs=realresnetModel.layers[-2].output)

resnet_model=models.Sequential()

resnet_model.add(realresnetModel)
resnet_model.add(layers.Conv2D(128,(3,3),padding='same',activation ='relu',input_shape=(224,224)))
resnet_model.add(layers.MaxPooling2D((2, 2)))
resnet_model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
resnet_model.add(layers.MaxPooling2D((2, 2)))
resnet_model.add(layers.Dropout(0.15))
resnet_model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
resnet_model.add(layers.Flatten())
resnet_model.add(Dense(16,activation='relu'))
resnet_model.add(layers.Dropout(0.2))
resnet_model.add(Dense(8,activation='relu'))
resnet_model.add(Dense(5,activation='softmax'))
resnet_model.summary()

In [ ]:
resnet_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [ ]:
testArray=np.array(testArray)
testLabel=np.array(testLabel)
valArray=np.array(valArray)
valLabel=np.array(valLabel)
history = resnet_model.fit(x=testArray,y=testLabel, epochs=50,validation_data=(valArray,valLabel))